In [1]:
#https://www.tensorflow.org/tutorials/quickstart/advanced 

import tensorflow as tf

print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Model

import tensorflow.keras as keras

import numpy as np

import matplotlib.pyplot as plt

#import tensorflow.keras.backend as K

2024-08-09 01:08:35.249517: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.10.0


In [2]:
# Universal parameters
batchSize = 1100 # batchSize = number of paths to be generated

d = 2 # dimension of forward process

batchSize_DNN = 1 #2 #10 # 20 50
Max_EPOCHS = 600
Mid_EPOCHS = 200
Min_EPOCHS = 200#60
lr = .001
seed = 0 #168
tf.random.set_seed(
    seed
)
tf.keras.utils.set_random_seed(
    seed
)

print('Number of paths (sample data) = ', batchSize)
print('batchSize = ', batchSize_DNN)
print('NumEpochs = ', Max_EPOCHS, Mid_EPOCHS, Min_EPOCHS)
print('Learning Rate = ', lr)



X_0 = 100 # initial price
print('initial price = ', X_0) 
V_0 = .15  # initial volatility
print('initial Vol = ', V_0)
K = 90 #80 #85 #90 #95 


mu = .08
print('mu = ', mu )
alpha = 5 
print('alpha = ', alpha)
m = 0 
print('m = ', m)
nu = 1
print('nu = ', nu)
a = nu * np.sqrt(2 * alpha)
print('nu * sqrt(2 alpha ) = ', a)
rho = -.2
print('rho = ', rho)
gamma = 1
print('gamma = ', gamma )
eta = .2 
print('eta = ', eta)
timeSteps = 10 #20 #50 
print('Total number of time steps = ', timeSteps )
r = 0.02 # .05
print('risk free rate r = ', r)
T = 1/4 # terminal time
print('T = ', T)
dt = T / timeSteps
print('dt = ', dt)

#strikes = [70,75,80,85,90,95,100,105,110,115]
# strikes = [i for i in range(70,116,2)]
#print('strikes = ', strikes)
#print('logMoney = ', np.array(strikes) / X_0)
# for loop here 
#ivValues = []
print('-------------Beginning computation-----------------')




Number of paths (sample data) =  1100
batchSize =  1
NumEpochs =  600 200 200
Learning Rate =  0.001
initial price =  100
initial Vol =  0.15
mu =  0.08
alpha =  5
m =  0
nu =  1
nu * sqrt(2 alpha ) =  3.1622776601683795
rho =  -0.2
gamma =  1
eta =  0.2
Total number of time steps =  10
risk free rate r =  0.02
T =  0.25
dt =  0.025
-------------Beginning computation-----------------


In [3]:
# First step is to train the deep neural network y^_{N-1} (a function of X_{N-1}, dW_{N-1}) by:
# input: X_{N-1}, dW_{N-1}
# output: y^_{N} = F(t_{N-1}, X_{N-1}, y^_{N-1}, z_{N-1}, dW_{N-1}) 
# (a custom layer is needed for a F transformation)
# minimizing quadratic loss: E( (y_{N} - y^_{N})^2 ), expectation is taken over all paths
# the best model y*_{N} = best(y^_{N}) is set to be the target of next deep neural network

In [4]:
def trainingData(seed): # = 168):
    tf.random.set_seed(seed)
    std_normal = tf.random.normal([batchSize,  # [x,   [number of paths,
                                   timeSteps + 1,  #  y, =  number of values in each path,            = tensor shape
                                   2],         #  z]    dim of each path 
                                  mean = 0.0,
                                  stddev = 1.0,
                                  dtype = tf.dtypes.float32,
                                  seed = seed,
                                  name = None)
    dW = std_normal * np.sqrt(dt)
    X = tf.Variable(tf.ones([batchSize, timeSteps + 1, 2], 
                            dtype = tf.dtypes.float32) * tf.constant([X_0, V_0]))
    for i in range(timeSteps):
        stockTerm1 = mu * X[:,i, 0] * dt
        stockTerm2 = (.7 / (np.pi) * (tf.math.atan(X[:,i, 1] - 1) + np.pi/2) + .03) * X[:, i,0] * dW[:,i,0] # sigma(Y_t)
        stockTerm3 = -r * X[:,i,0] * dt
        X[:, i+1, 0].assign(X[:,i, 0] + stockTerm1 + stockTerm2 + stockTerm3)
        volTerm1 = alpha * (m - X[:,i,1]) * dt
        volTerm2 = a * (rho *dW[:,i,0] + np.sqrt(1 - rho ** 2)* dW[:,i,1])
        X[:, i+1, 1].assign(X[:, i, 1] + volTerm1 + volTerm2) 
        
       # X = X[:, i+1, :].assign(X[:, i, :] + r*X[:, i, :]*dt + sigma*X[:, i, :]*dW[:, i, :])
    return X, dW

    
#terminal condition
def fun_g(X, K, t):
    # print(X.shape)
    g = tf.math.maximum(K * tf.math.exp((-r * (t/timeSteps) * T)) - X, 0)
    return -1 * g



 #X2=X[:,t, 1]
# driver 1    
def fun_f(t, X1, X2, z1, z2):
    sigma = ((.7 / (np.pi)) * (tf.math.atan(X2 - 1) + np.pi/2) + .03)
    term1 = -1 * z1 * ((mu - r) / sigma) 
    term2 = -1 * (1/(2 * gamma)) * ((-(mu - r) / sigma) **2)
    term3 =  (1/(2 * gamma)) * (gamma / 2) * z2 ** 2
    term4 = -1 * (1/(2 * gamma)) * eta * ((mu - r) / sigma) * z2

    regular = term1 + term2 + term3 + term4
    
    return regular
#print(fun_f(1,10,.5,1,2))
# driver 2
def fun_f_bar(t, X1, X2, z1, z2):
    z1 = -1 * z1
    z2 = -1 * z2
    sigma = (.7 / (np.pi) * (tf.math.atan(X2 - 1) + np.pi/2) + .03)
    term1 = -1 * z1 * ((mu - r) / sigma) 
    term2 = -1 * (1/(2 * gamma)) * ((-(mu - r) / sigma) **2)
    term3 =  (1/(2 * gamma)) * (gamma / 2) * z2 ** 2
    term4 = -1 * (1/(2 * gamma)) * eta * ((mu - r) / sigma) * z2

    regular = term1 + term2 + term3 + term4
    
    return -1 * regular

# function F (the estimation of y_{i+1} given t_i, X_i, y_i, z_i, dW_i)
def fun_F(t, X1, X2, y, z1, z2, dW1, dW2):
    F = y - fun_f(t, X1, X2, z1, z2) * dt + z1 * dW1 + z2 * dW2
    return F



# function F (the estimation of y_{i+1} given t_i, X_i, y_i, z_i, dW_i)
def fun_F_bar(t, X1, X2, y, z1, z2, dW1, dW2):
    F = y - fun_f_bar(t, X1, X2, z1, z2)*dt + z1 * dW1 + z2 * dW2
    return F



In [5]:
X, dW = trainingData(seed)
print('sample path = ', X[1,:,:])
# print('sample brownian motion = ', dW[0,:,:])
print('sample BM mean: ', tf.math.reduce_mean(dW), '| variance: ', tf.math.reduce_variance(dW))
                    

sample path =  tf.Tensor(
[[ 1.00000000e+02  1.50000006e-01]
 [ 1.00554573e+02 -2.82164752e-01]
 [ 1.02307930e+02 -1.57119930e-02]
 [ 9.80057449e+01 -2.48888656e-01]
 [ 1.04667213e+02 -3.32593203e-01]
 [ 1.06612518e+02 -3.46326292e-01]
 [ 1.06523796e+02 -4.39549088e-02]
 [ 1.08641037e+02 -5.97284615e-01]
 [ 1.10312263e+02  2.68377781e-01]
 [ 1.19024925e+02  5.41483462e-01]
 [ 1.06819839e+02  5.83199024e-01]], shape=(11, 2), dtype=float32)
sample BM mean:  tf.Tensor(0.001961416, shape=(), dtype=float32) | variance:  tf.Tensor(0.024696153, shape=(), dtype=float32)


2024-08-09 01:08:38.776216: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = Flatten()
        
        self.d1_y = keras.layers.Dense(10+d, activation='relu', input_shape=(2,))
        self.d2_y = keras.layers.Dense(10+d, activation='relu')
        self.d3_y = keras.layers.Dense(1)
        
        self.d4_z = keras.layers.Dense(10+d, activation='relu', input_shape=(2,))
        self.d5_z = keras.layers.Dense(10+d, activation='relu')
        self.d6_z = keras.layers.Dense(2)
        
    def call(self, y):
        
        y_0 = tf.reshape(y[:,0], (batchSize_DNN,1), name=None) ## X
        y_1 = tf.reshape(y[:,1], (batchSize_DNN,1), name=None) ## V
        y_2 = tf.reshape(y[:,2], (batchSize_DNN,1), name=None) ## dW1
        y_3 = tf.reshape(y[:,3], (batchSize_DNN,1), name=None) ## dW2
        
        y_raw = tf.concat([y_0,y_1], 1)
        
        y = self.flatten(y_raw)
        y = self.d1_y(y)
        y = self.d2_y(y)
        y = self.d3_y(y)
        
        y_export = y
        
        z = self.flatten(y_raw)
        z = self.d4_z(z)
        z = self.d5_z(z)
        z = self.d6_z(z)
        
        z_export = z
        z_1 = z[:,0]
        z_2 = z[:,1]
       
        p = fun_F(t = time, # not useful
                  X1 = y_0,
                  X2 = y_1,
                  y = y,
                  z1 = z_1,
                  z2 = z_2,
                  dW1 = y_2,
                  dW2 = y_3)
             # t is a placeholder only, not useful, 
             # X is the first column of input data (input data = (X,dW))
             # y is the output of the last dense layer (1st dim of output (Y,z))
             # z is the sigma^T*D_x*y, where D_x*y is the numerical differentiation of y
             # dW is the second column of input data (input data = (X,dW))

        return p, y_export, z_export # dimensions (1, 1, 2)

In [7]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

In [8]:
train_MSE = tf.keras.metrics.MeanSquaredError(name='train_MSE')
test_MSE = tf.keras.metrics.MeanSquaredError(name='test_MSE')

In [9]:
@tf.function
def train_step(x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training = True)[0]
        loss = loss_object(y_train, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_MSE(y_train, predictions)

In [10]:
@tf.function
def test_step(x_test, y_test):
    predictions = model(x_test, training = False)[0]
    #t_loss = loss_object(y_test, predictions)

    test_MSE(y_test, predictions)

In [11]:
X, dW = trainingData(seed)

In [12]:
# Create an instance of the model 
model = MyModel() 

print("Strike = ", K)
   #DO TERMINAL TIME SEPARATELY 
for time in range(timeSteps-1, -1, -1):
    #model.set_weights(initial_weights)
    
   

    time_plus1 = time + 1
    path_ds = tf.concat([X, dW], axis = 2)
    path_ds = path_ds[:,time,:]
    #print("input data shape:")
    #print(path_ds.shape)
    #print(fun_g(X[:,time_plus1,0], K = K, t = time_plus1).shape)

    if time == timeSteps-1:
        payoff_timeplus1 = fun_g(X[:,time_plus1,0], K = K, t = time_plus1)
    else :
        payoff_timeplus1 = np.loadtxt("Buy-BSDE_predictions_"+str(time_plus1)+"K_"+str(K)+".csv", delimiter = ",", dtype = 'float32')
        payoff_timeplus1 = payoff_timeplus1.reshape((batchSize, 1))
     
   
    #print("output data shape:")
    #print(payoff_timeplus1.shape)
    (x_train, y_train), (x_test, y_test) = (path_ds[0:1000,:], 
                       payoff_timeplus1[0:1000]), (path_ds[1000:1100,:], 
                                                payoff_timeplus1[1000:1100])
    
    
    

    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(batchSize_DNN)

    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batchSize_DNN)#.shuffle(100).batch(batchSize_DNN)

    if time == timeSteps-1:
        EPOCHS = Max_EPOCHS
    elif time < 6:
        EPOCHS = Mid_EPOCHS
    else :
        EPOCHS = Min_EPOCHS    
    #EPOCHS = 60
    for epoch in range(EPOCHS):
        train_MSE.reset_states()
        test_MSE.reset_states()
        
        for x_train, y_train in train_ds:
            train_step(x_train, y_train)
        
        for x_test, y_test in test_ds:
            test_step(x_test, y_test)
        
        if (epoch + 1) %100 == 0 :# and verbose == True:
            print(
            f'Epoch {epoch + 1}, '
            f'MSE: {train_MSE.result() * 100}, '
            f'Test MSE: {test_MSE.result() * 100}'
               )

    predictions = model.predict(path_ds, batch_size = batchSize_DNN)
    predictions_time = predictions[0].reshape((batchSize,1))
    predictions_time_y = predictions[1]
    

    print("Time = ", time)
    print(test_MSE(payoff_timeplus1, predictions_time))

    print(np.mean(payoff_timeplus1))
    print(np.mean(predictions_time))
    print(np.mean(predictions_time_y))
    
    np.savetxt("Buy-BSDE_predictions_"+str(time)+"K_"+str(K)+".csv", predictions_time_y, delimiter = ",")

    model.reset_states()
    
    
   
    
    
    
  

Strike =  90
Epoch 100, MSE: 237.5840606689453, Test MSE: 374.44683837890625
Epoch 200, MSE: 130.1310577392578, Test MSE: 126.58877563476562
Epoch 300, MSE: 92.46658325195312, Test MSE: 102.16560363769531
Epoch 400, MSE: 107.19892883300781, Test MSE: 77.99032592773438
Epoch 500, MSE: 76.2447509765625, Test MSE: 89.78536224365234
Epoch 600, MSE: 62.47478103637695, Test MSE: 99.07984161376953
1100/1100 [==============================] - 1s 467us/step
Time =  9
tf.Tensor(0.9895599, shape=(), dtype=float32)
-1.1946193
-1.4462891
-1.5175321
Epoch 100, MSE: 57.640411376953125, Test MSE: 32.16469955444336
Epoch 200, MSE: 55.792076110839844, Test MSE: 33.21193313598633
1100/1100 [==============================] - 1s 479us/step
Time =  8
tf.Tensor(0.3684286, shape=(), dtype=float32)
-1.5175321
-1.4717613
-1.434841
Epoch 100, MSE: 60.684226989746094, Test MSE: 42.40276336669922
Epoch 200, MSE: 60.59519577026367, Test MSE: 31.488237380981445
1100/1100 [==============================] - 1s 479us/s

In [15]:
print(np.mean(predictions_time_y))

-0.60750467


In [16]:
BSDE = np.loadtxt("Buy-BSDE_predictions_"+str(0)+"K_"+str(K)+".csv", delimiter = ",", dtype = 'float32')
BSDE0payoff = np.loadtxt("BSDE0predictions_"+str(0)+"_y.csv", delimiter = ",") 
Price = BSDE0payoff - BSDE
print('Buyer Price = ', np.mean(Price))
print('variance = ', np.var(Price))

Buyer Price =  0.5957556068897247
variance =  0.0


In [19]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  36        
                                                                 
 dense_1 (Dense)             multiple                  156       
                                                                 
 dense_2 (Dense)             multiple                  13        
                                                                 
 dense_3 (Dense)             multiple                  36        
                                                                 
 dense_4 (Dense)             multiple                  156       
                                                                 
 dense_5 (Dense)             multiple                  26 